In [7]:
#Import libraries
import pandas as pd
import os
import numpy as np


# Ensure "processed" folder exists
os.makedirs("../data/processed", exist_ok=True)

In [8]:
data_path = "../data/raw"
assets = {}  # Dictionary to store each asset

# Load each CSV file with appropriate settings
for file in os.listdir(data_path):
    file_path = os.path.join(data_path, file)
    name = file.replace(".csv", "")
    df = pd.read_csv(
        file_path,
        header=0,                 # Use the first row as header
        parse_dates=["Date"],     # Parse 'Date' column as datetime
        index_col="Date"          # Set 'Date' column as index
    )
    assets[name] = df

# Quick check: print number of rows for each asset
for name, df in assets.items():
    print(f"{name}: {df.shape[0]} rows, {df.shape[1]} columns")
    print(df.head())

FTSE100: 2525 rows, 5 columns
                  Close         High          Low         Open     Volume
Date                                                                     
02/01/2015  6547.799805  6607.899902  6510.600098  6566.100098  367562200
05/01/2015  6417.200195  6576.700195  6404.500000  6547.799805  728001100
06/01/2015  6366.500000  6452.700195  6328.600098  6417.200195  769463100
07/01/2015  6419.799805  6459.700195  6366.500000  6366.500000  688211500
08/01/2015  6570.000000  6580.799805  6419.799805  6419.799805  882740200
GBPUSD: 2606 rows, 5 columns
               Close      High       Low      Open  Volume
Date                                                      
01/01/2015  1.558094  1.558094  1.558094  1.558094       0
02/01/2015  1.557972  1.558458  1.538746  1.558021       0
05/01/2015  1.528491  1.532004  1.520612  1.528608       0
06/01/2015  1.525832  1.527370  1.515795  1.525832       0
07/01/2015  1.513798  1.515611  1.506796  1.513638       0
SP500: 251

In [9]:
# Find the intersection of dates across all datasets
common_dates = assets[next(iter(assets))].index
for df in assets.values():
    common_dates = common_dates.intersection(df.index)

# Filter each dataset to only include common dates
for name in assets:
    assets[name] = assets[name].loc[common_dates]

# Confirm alignment by printing the first few rows of each asset
for name, df in assets.items():
    print(f"{name}: {df.shape[0]} rows, {df.shape[1]} columns")
    print(df.head())

FTSE100: 2466 rows, 5 columns
                  Close         High          Low         Open     Volume
Date                                                                     
02/01/2015  6547.799805  6607.899902  6510.600098  6566.100098  367562200
05/01/2015  6417.200195  6576.700195  6404.500000  6547.799805  728001100
06/01/2015  6366.500000  6452.700195  6328.600098  6417.200195  769463100
07/01/2015  6419.799805  6459.700195  6366.500000  6366.500000  688211500
08/01/2015  6570.000000  6580.799805  6419.799805  6419.799805  882740200
GBPUSD: 2466 rows, 5 columns
               Close      High       Low      Open  Volume
Date                                                      
02/01/2015  1.557972  1.558458  1.538746  1.558021       0
05/01/2015  1.528491  1.532004  1.520612  1.528608       0
06/01/2015  1.525832  1.527370  1.515795  1.525832       0
07/01/2015  1.513798  1.515611  1.506796  1.513638       0
08/01/2015  1.510757  1.511899  1.503624  1.510711       0
SP500: 246

In [10]:
# Keep only Close prices
close_prices = {}

for name, df in assets.items():
    close_prices[name] = df[["Close"]].copy()

# Quick check
for name, df in close_prices.items():
    print(name)
    print(df.head())

FTSE100
                  Close
Date                   
02/01/2015  6547.799805
05/01/2015  6417.200195
06/01/2015  6366.500000
07/01/2015  6419.799805
08/01/2015  6570.000000
GBPUSD
               Close
Date                
02/01/2015  1.557972
05/01/2015  1.528491
06/01/2015  1.525832
07/01/2015  1.513798
08/01/2015  1.510757
SP500
                  Close
Date                   
02/01/2015  2058.199951
05/01/2015  2020.579956
06/01/2015  2002.609985
07/01/2015  2025.900024
08/01/2015  2062.139893
UK_Gov_Bonds
                 Close
Date                  
02/01/2015  646.342041
05/01/2015  632.999084
06/01/2015  628.319031
07/01/2015  633.695984
08/01/2015  647.636597


In [11]:
combined_close = pd.concat(
    close_prices.values(),
    axis=1,
    keys=close_prices.keys()
)

# Flatten column names
combined_close.columns = combined_close.columns.droplevel(1)

# Quick check
print(combined_close.shape)
print(combined_close.head())

(2466, 4)
                FTSE100    GBPUSD        SP500  UK_Gov_Bonds
Date                                                        
02/01/2015  6547.799805  1.557972  2058.199951    646.342041
05/01/2015  6417.200195  1.528491  2020.579956    632.999084
06/01/2015  6366.500000  1.525832  2002.609985    628.319031
07/01/2015  6419.799805  1.513798  2025.900024    633.695984
08/01/2015  6570.000000  1.510757  2062.139893    647.636597


In [ ]:
#Returns Metrics
# 1. Daily Returns
daily_returns = combined_close.pct_change().dropna()

# 2. Cumulative Returns
cumulative_returns = (1 + daily_returns).cumprod() - 1

# 3. Log Returns (optional)
log_returns = np.log(combined_close / combined_close.shift(1)).dropna()

# Quick check
print("Daily Returns:\n", daily_returns.head())
print("\nCumulative Returns:\n", cumulative_returns.head())
print("\nLog Returns:\n", log_returns.head())

Daily Returns:
              FTSE100    GBPUSD     SP500  UK_Gov_Bonds
Date                                                  
05/01/2015 -0.019946 -0.018923 -0.018278     -0.020644
06/01/2015 -0.007901 -0.001739 -0.008893     -0.007393
07/01/2015  0.008372 -0.007887  0.011630      0.008558
08/01/2015  0.023396 -0.002009  0.017888      0.021999
09/01/2015 -0.010487 -0.001298 -0.008404     -0.009686

Cumulative Returns:
              FTSE100    GBPUSD     SP500  UK_Gov_Bonds
Date                                                  
05/01/2015 -0.019946 -0.018923 -0.018278     -0.020644
06/01/2015 -0.027689 -0.020629 -0.027009     -0.027885
07/01/2015 -0.019549 -0.028353 -0.015693     -0.019566
08/01/2015  0.003390 -0.030306  0.001914      0.002003
09/01/2015 -0.007132 -0.031564 -0.006506     -0.007703

Log Returns:
              FTSE100    GBPUSD     SP500  UK_Gov_Bonds
Date                                                  
05/01/2015 -0.020147 -0.019104 -0.018447     -0.020860
06/01/2015 -

In [13]:
#Rolling Volatility (30-day window)
rolling_vol = daily_returns.rolling(window=30).std()

#Maximum Drawdown
def max_drawdown(cum_returns):
    """
    Calculate maximum drawdown for a cumulative return series.
    Returns a series with the max drawdown at each point.
    """
    rolling_max = cum_returns.cummax()
    drawdown = (cum_returns - rolling_max) / rolling_max
    return drawdown.min()  # maximum drawdown is the minimum value

#Compute max drawdown for each asset
max_dd = cumulative_returns.apply(max_drawdown)

#Correlation Matrix of Daily Returns
correlation_matrix = daily_returns.corr()

#Quick Check
print("Rolling Volatility (last 5 rows):")
print(rolling_vol.tail())

print("\nMaximum Drawdowns:")
print(max_dd)

print("\nCorrelation Matrix:")
print(correlation_matrix)

Rolling Volatility (last 5 rows):
             FTSE100    GBPUSD     SP500  UK_Gov_Bonds
Date                                                  
23/12/2024  0.005757  0.004804  0.007533      0.005821
24/12/2024  0.005683  0.004758  0.007802      0.005728
27/12/2024  0.005191  0.004740  0.008072      0.005244
30/12/2024  0.005243  0.004451  0.008303      0.005281
31/12/2024  0.005289  0.004448  0.008268      0.005261

Maximum Drawdowns:
FTSE100         -8.175604
GBPUSD         -16.888016
SP500          -17.632201
UK_Gov_Bonds   -13.230033
dtype: float64

Correlation Matrix:
               FTSE100    GBPUSD     SP500  UK_Gov_Bonds
FTSE100       1.000000 -0.000588  0.529461      0.987963
GBPUSD       -0.000588  1.000000  0.025280     -0.004839
SP500         0.529461  0.025280  1.000000      0.528087
UK_Gov_Bonds  0.987963 -0.004839  0.528087      1.000000


In [17]:
output_folder = r"E:\uk-financial-risk-analysis\data\processed"

# Save files to data/processed
daily_returns.to_csv(f'{output_folder}/daily_returns.csv')
cumulative_returns.to_csv(f'{output_folder}/cumulative_returns.csv')
rolling_vol.to_csv(f'{output_folder}/rolling_volatility.csv')
max_dd.to_frame(name='Max_Drawdown').to_csv(f'{output_folder}/max_drawdown.csv')
correlation_matrix.to_csv(f'{output_folder}/correlation_matrix.csv')